In [1]:
%%pyspark
df = spark.read.load('abfss://files@datalake5bdznl9.dfs.core.windows.net/products/products.csv', format='csv'
, header=True
)
display(df.limit(10))

StatementMeta(spark5bdznl9, 0, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 267b73fe-2699-441c-8bf8-1c38a9bc9eff)

In [2]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").save(delta_table_path)

StatementMeta(spark5bdznl9, 0, 3, Finished, Available)

In [3]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

StatementMeta(spark5bdznl9, 0, 4, Finished, Available)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [4]:
 new_df = spark.read.format("delta").load(delta_table_path)
 new_df.show(10)

StatementMeta(spark5bdznl9, 0, 5, Finished, Available)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [5]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

StatementMeta(spark5bdznl9, 0, 6, Finished, Available)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [6]:
 deltaTable.history(10).show(20, False, True)

StatementMeta(spark5bdznl9, 0, 7, Finished, Available)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                         
 timestamp           | 2023-10-28 01:44:38                                                                                                                                       
 userId              | null                                                                                                                                                      
 userName            | null                                                                                                                                                      
 operation           | UPDATE                                                                                 

In [7]:
 spark.sql("CREATE DATABASE AdventureWorks")
 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

StatementMeta(spark5bdznl9, 0, 8, Finished, Available)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|ProductID                   |string                                                                 |       |
|ProductName                 |string                                                                 |       |
|Category                    |string                                                                 |       |
|ListPrice                   |string                                                                 |       |
|                            |                                                                       |       |
|# Partitioning              |                                                                       |       |
|

In [8]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsExternal;

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [9]:
 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

StatementMeta(spark5bdznl9, 0, 11, Finished, Available)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |       |
|ProductName                 |string                                                                                                                          |       |
|Category                    |string                                                                                                                          | 

In [10]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsManaged;

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [11]:
 %%sql

 USE AdventureWorks;

 SHOW TABLES;

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 2 rows and 3 fields>

In [12]:
 %%sql

 USE AdventureWorks;

 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
 %%sql

 USE AdventureWorks;

 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
 %%sql

 USE AdventureWorks;

 SELECT * FROM Products;

StatementMeta(, 0, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [15]:
 from notebookutils import mssparkutils
 from pyspark.sql.types import *
 from pyspark.sql.functions import *

 # Create a folder
 inputPath = '/data/'
 mssparkutils.fs.mkdirs(inputPath)

 # Create a stream that reads data from the folder, using a JSON schema
 jsonSchema = StructType([
 StructField("device", StringType(), False),
 StructField("status", StringType(), False)
 ])
 iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

 # Write some event data to the folder
 device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
 mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
 print("Source stream created...")

StatementMeta(spark5bdznl9, 0, 23, Finished, Available)

Source stream created...


In [16]:
 # Write the stream to a delta table
 delta_stream_table_path = '/delta/iotdevicedata'
 checkpointpath = '/delta/checkpoint'
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")

StatementMeta(spark5bdznl9, 0, 24, Finished, Available)

Streaming to delta sink...


In [17]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)

StatementMeta(spark5bdznl9, 0, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, bf2e2be3-6a18-4da5-9da9-cad63fecdcac)

In [19]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(spark5bdznl9, 0, 27, Finished, Available)

DataFrame[]

In [20]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(spark5bdznl9, 0, 28, Finished, Available)

<Spark SQL result set with 9 rows and 2 fields>

In [21]:
 # Add more data to the source stream
 more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(spark5bdznl9, 0, 29, Finished, Available)

True

In [22]:
 %%sql

 SELECT * FROM IotDeviceData;

StatementMeta(spark5bdznl9, 0, 30, Finished, Available)

<Spark SQL result set with 16 rows and 2 fields>

In [23]:
 deltastream.stop()

StatementMeta(spark5bdznl9, 0, 31, Finished, Available)